# Overview

This notebook includes prototypical code for the operator overview write-up. It does not depend on any exisiting code in DiffEqOperators, but it should be easy to modify the current codebase to achieve the same functionality.

To make things simpler, everything is assumed to be time-invariant (so no `update_coefficients!`) and the datatype is `Float64`. I will also always use the out-of-place convention (i.e. `*` instead of `A_mul_B!`).

In [1]:
import Base: +, *

abstract type DiffEqOperator end
abstract type DiffEqLinearOperator <: DiffEqOperator end

# 1. Abstract Operator Interface

Below is a simplified operator interface used in my new interface draft. Basically we want to express lazy addition and multiplication of linear operators naturally using `+` and `*`. The `as_array` interface returns the most suitable (dense/sparse) representation of the underlying operator as an `AbstractMatrix` (or should we treat this as a type conversion and just use `AbstractMatrix`?).

The affine operator is defined in such a way that arithmetic on them and linear operators always yield an `DiffEqAffineOperator`.

In [2]:
struct DiffEqArrayOperator <: DiffEqLinearOperator
    A::AbstractMatrix{Float64}
end

*(L::DiffEqArrayOperator, x::Vector{Float64}) = L.A * x
as_array(L::DiffEqArrayOperator) = L.A

struct DiffEqOperatorCombination <: DiffEqLinearOperator
    ops::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorCombination, x::Vector{Float64}) = sum(op -> op*x, L.ops)
as_array(L::DiffEqOperatorCombination) = sum(as_array, L.ops)

struct DiffEqOperatorComposition <: DiffEqLinearOperator
    ops::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorComposition, x::Vector{Float64}) = foldl((u, op) -> op*u, x, L.ops)
as_array(L::DiffEqOperatorComposition) = prod(as_array, reverse(L.ops))

# Arithmetic
# op + op
+(L1::DiffEqOperatorCombination, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination((L1.ops..., L2.ops...))
+(L1::DiffEqOperatorCombination, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1.ops..., L2))
+(L1::DiffEqLinearOperator, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination((L1, L2.ops...))
+(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1,L2))

# op * op
# Note the application order
*(L1::DiffEqOperatorComposition, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.ops..., L1.ops...))
*(L1::DiffEqLinearOperator, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.ops..., L1))
*(L1::DiffEqOperatorComposition, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1.ops...))
*(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1));

In [3]:
# Affine Operator
struct DiffEqAffineOperator <: DiffEqOperator
    A::DiffEqOperator
    b::Vector{Float64}
end

*(L::DiffEqAffineOperator, x::Vector{Float64}) = L.A * x + L.b

# Arithmetic on affine operators
+(L1::DiffEqLinearOperator, L2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 + L2.A, L2.b)
+(L1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(L1.A + L2, L1.b)
+(L1::DiffEqAffineOperator, L2::DiffEqAffineOperator) = DiffEqAffineOperator(L1.A + L2.A, L1.b + L2.b)
*(L1::DiffEqLinearOperator, L2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 * L2.A, L1 * L2.b)
*(L1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(L1.A * L2, L1.b)
*(L1::DiffEqAffineOperator, L2::DiffEqAffineOperator) = DiffEqAffineOperator(L1.A * L2.A, L1.A * L2.b + L1.b);

# 2. Operators on the interior (stencil convolution)

The 2nd-order central difference approximation to $\partial_x^2/2$ and the 1st-order upwind approximation to $\partial_x$ are included. The general case can be modified from the stencil convolution code in DiffEqOperators.

The upwind operator is implemented differently from DiffEqOperators, where the direction at each point is stored. Here only the left/right operator is constructed and we interpret

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

This is the approach used in the write-up. Though maybe not as efficient, this is clearer both conceptually and from a coding standpoint.

In [4]:
struct DiffusionOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
end

*(L::DiffusionOperator, x::Vector{Float64}) = [x[i] + x[i+2] - 2*x[i+1] for i in 1:L.m] / L.dx^2
as_array(L::DiffusionOperator) = spdiagm((ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) / L.dx^2;

In [5]:
struct DriftOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
    direction::Bool # true = right, false = left
end

function *(L::DriftOperator, x::Vector{Float64})
    if L.direction # right drift
        [x[i+1] - x[i] for i in 1:L.m] / L.dx
    else # left drift
        [x[i+2] - x[i+1] for i in 1:L.m] / L.dx
    end
end
function as_array(L::DriftOperator)
    if L.direction # right drift
        spdiagm((-ones(L.m), ones(L.m), zeros(L.m)), (0,1,2)) / L.dx
    else # left drift
        spdiagm((zeros(L.m), -ones(L.m), ones(L.m)), (0,1,2)) / L.dx
    end
end;

# 3. Boundary Maps (The $Q$ Operator)

(I'm calling the $Q$ operators the "boundary maps" temporarily, but I guess a better name should be used?)

A generic $Q$ from the generic boundary condition $Bu = b$ can be a bit difficult to implement, but the simple case of Dirichlet/Neumann BC is easy to handle.

It should be easy to modify `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` to incorporate boundaries that are absorbing at one end and reflecting at another.

The non-zero Dirichlet/Neumann boundary maps do not have their own type. Instead we construct the `Q` operator as an affine map, with `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` its linear part.

In [6]:
struct AbsorbingBoundaryMap <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::AbsorbingBoundaryMap, x::Vector{Float64}) = [0.0; x; 0.0]
as_array(Q::AbsorbingBoundaryMap) = sparse([zeros(Q.m)'; eye(Q.m); zeros(Q.m)'])

struct ReflectingBoundaryMap <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::ReflectingBoundaryMap, x::Vector{Float64}) = [x[1]; x; x[end]]
as_array(Q::ReflectingBoundaryMap) = sparse([1.0 zeros(Q.m-1)'; eye(Q.m); zeros(Q.m-1)' 1.0]);

In [7]:
function Dirichlet_boundary_map(m::Int, bl::Float64, br::Float64)
    # y[1] = bl, y[end] = br
    Qa = AbsorbingBoundaryMap(m)
    Qb = [bl; zeros(m); br]
    DiffEqAffineOperator(Qa, Qb)
end

function Neumann_boundary_map(m::Int, dx::Float64, bl::Float64, br::Float64)
    # (y[2] - y[1])/dx = bl, (y[end] - y[end-1])/dx = br
    Qa = ReflectingBoundaryMap(m)
    Qb = [-bl*dx; zeros(m); br*dx]
    DiffEqAffineOperator(Qa, Qb)
end;

Examples of non-zero BC:

In [8]:
dx = 1.0
u = [1.,2.,3.,4.]
QD = Dirichlet_boundary_map(4, 10.0, 20.0)
QN = Neumann_boundary_map(4, dx, 1.0, 2.0);

In [9]:
QD * u

6-element Array{Float64,1}:
 10.0
  1.0
  2.0
  3.0
  4.0
 20.0

In [10]:
QN * u

6-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0
 4.0
 6.0

# 4. Constructing operators for the Fokker-Planck equations

The most general case is in section 3.5 of the write-up:

$$ \mathcal{L} = \mu(x)\partial_x + \frac{\sigma(x)^2}{2}\partial_{xx} $$

where the drift term is discretized using upwind operators as described in section 2:

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

The discretized operators are expressed as the composition of an interior stencil convolution operator `A` and boundary map `Q` (they are generally affine). The drift and diffusion coefficients can be expressed as diagonal matrices (wrapped in a `DiffEqArrayOperator`).

The script below can be modified to represent each of the scenarios described in secition 3 of the write-up (except the mixed-BC case).

In [11]:
# The grid
N = 10
dx = 1.0
xs = collect(1:N) * dx # interior nodes

# Interior operators
A1p = DriftOperator(dx, N, true)
A1m = DriftOperator(dx, N, false)
A2 = DiffusionOperator(dx, N)

# Boundary operators
Q = AbsorbingBoundaryMap(N)
# Q = Neumann_boundary_map(N, dx, 1.0, 2.0)

# Coefficients
mu = rand(N)
mup = [mu[i] > 0.0 ? mu[i] : 0.0 for i in 1:N]
mum = [mu[i] < 0.0 ? mu[i] : 0.0 for i in 1:N]
sigma = rand(N) + 1.0

# Construct L
L1 = DiffEqArrayOperator(Diagonal(mup)) * A1p + DiffEqArrayOperator(Diagonal(mum)) * A1m
L2 = DiffEqArrayOperator(Diagonal(sigma.^2 / 2)) * A2
L = (L1 + L2) * Q

DiffEqOperatorComposition((AbsorbingBoundaryMap(10), DiffEqOperatorCombination((DiffEqOperatorComposition((DriftOperator(1.0, 10, true), DiffEqArrayOperator([0.123526 0.0 … 0.0 0.0; 0.0 0.77393 … 0.0 0.0; … ; 0.0 0.0 … 0.672812 0.0; 0.0 0.0 … 0.0 0.751455]))), DiffEqOperatorComposition((DriftOperator(1.0, 10, false), DiffEqArrayOperator([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))), DiffEqOperatorComposition((DiffusionOperator(1.0, 10), DiffEqArrayOperator([1.32813 0.0 … 0.0 0.0; 0.0 0.59207 … 0.0 0.0; … ; 0.0 0.0 … 1.87849 0.0; 0.0 0.0 … 0.0 1.22985])))))))

(The standard printout for the composed operator is a bit messy. Should probably implement `Base.show` for the composition types)

In [12]:
# Solve the HJBE (rI - L)u = x
r = 0.05
if isa(L, DiffEqAffineOperator)
    LHS = r*speye(N) - as_array(L.A)
    RHS = xs + L.b
else
    LHS = r*speye(N) - as_array(L)
    RHS = xs
end
u = LHS \ RHS

10-element Array{Float64,1}:
 32.1917
 61.8482
 54.584 
 47.3014
 46.0182
 43.7368
 38.5641
 31.9659
 22.1972
 11.7271